In [1]:
import pandas as pd
import numpy as np

# TWEETS

In [2]:
# load in tweet df
# https://drive.google.com/file/d/1vAkSbl2YtPau2r7LUMPaSEuNxqZ2Fxlu/view?usp=sharing

# df_tweets = pd.read_csv('/Users/Mary/Desktop/NLP/Group Project/final_cleaned_tweets.csv')
df_tweets = pd.read_csv('/Users/creeg/Downloads/final_cleaned_tweets.csv')
del df_tweets["Unnamed: 0"] # delete unnecessary index column from last write


C:\Users\creeg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# df should have 1276597 records
len(df_tweets)

1276597

### tweet cleaning

df should return: 
- 261010001 total characters before cleaning across 1276597 tweets
- 162741884 total characters after cleaning across 951466 tweets

In [4]:
print(str(sum(df_tweets['Text'].map(len)))+ ' total characters before cleaning across '+str(len(df_tweets))+' tweets')


# make text lowercase
df_tweets['Text'] = df_tweets['Text'].str.lower() #.astype('unicode')

# remove apostrophes, newline, non-ascii characters
df_tweets['Text'] = df_tweets['Text'].str.replace("\’", "").str.strip() 
df_tweets['Text'] = df_tweets['Text'].str.replace("\''", "").str.strip() 
df_tweets['Text'] = df_tweets['Text'].str.replace("\n", "").str.strip() 
df_tweets.Text.str.encode('ascii', 'ignore').str.decode('ascii')


### remove emoji
import re

regex_pattern = re.compile(pattern = "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       "]+", flags = re.UNICODE)

df_tweets['Text'] = df_tweets['Text'].str.replace(regex_pattern, "").str.strip() 


# remove twitter handles (words in tweet that start with "@")
# NOTE: this line must be above regext to remove all punctuation!
df_tweets['Text'] = df_tweets['Text'].apply(lambda x: re.sub('(?:\s)@[^, ]*', '', x))
df_tweets['Text'] = df_tweets['Text'].apply(lambda x: re.sub('@[^, ]*', '', x))


# remove all punctuation except asterisk
import string

remove = string.punctuation
remove = remove.replace("*", "") # don't remove hyphens
punct_pattern = r"[{}]".format(remove) # create the pattern

df_tweets['Text'] = df_tweets['Text'].str.replace(punct_pattern, "").str.strip() 

# remove numbers
nums = pattern = r'[0-9]'
df_tweets['Text'] = df_tweets['Text'].str.replace(nums, "").str.strip() 

# remove non-english characters
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

df_tweets = df_tweets[df_tweets['Text'].apply(isEnglish)] 



print(str(sum(df_tweets['Text'].map(len)))+ ' total characters after cleaning across '+str(len(df_tweets))+' tweets')

261010001 total characters before cleaning across 1276597 tweets


C:\Users\creeg\AppData\Local\Temp/ipykernel_15492/183405973.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df_tweets['Text'] = df_tweets['Text'].str.replace("\’", "").str.strip()
C:\Users\creeg\AppData\Local\Temp/ipykernel_15492/183405973.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  df_tweets['Text'] = df_tweets['Text'].str.replace(punct_pattern, "").str.strip()
C:\Users\creeg\AppData\Local\Temp/ipykernel_15492/183405973.py:44: FutureWarning: The default value of regex will change from True to False in a future version.
  df_tweets['Text'] = df_tweets['Text'].str.replace(nums, "").str.strip()


162741884 total characters after cleaning across 951466 tweets


In [5]:
df_tweets.head()

,Datetime,Tweet Id,Text,Username,hashtag
0,2022-03-23 23:50:07+00:00,1506780360834748419,off to a great startthesevenhusbandsofevelynhu...,actuallydana,#booktok
1,2022-03-23 23:15:05+00:00,1506771544600719367,enoch by jarrod edge christian scifiprophecy o...,IAN_AuthorPromo,#booktok
2,2022-03-23 23:00:01+00:00,1506767752152985610,soooooo im on tiktok httpstcodyaeldna now ive ...,LovinSunshine97,#booktok
3,2022-03-23 22:52:17+00:00,1506765808218820610,guysi just hit k on my booktok account bookto...,adorkablebooks,#booktok
5,2022-03-23 21:44:11+00:00,1506748670095548422,im officially on twitter books booktok,themelissablair,#booktok


In [6]:
# add month year column and check how many tweets each month has
df_tweets['tweet_yr_mth'] = df_tweets['Datetime'].str[:7]
df_tweets.groupby(['tweet_yr_mth']).size()

tweet_yr_mth
 #Font         1
 #books        1
 #flowe        1
 #newgi        1
 @anuja        1
#MadeWi        1
#audibl        1
#book #        4
#rsamen        1
2021-03    21061
2021-04    85257
2021-05    81819
2021-06    84368
2021-07    77629
2021-08    79570
2021-09    76929
2021-10    72203
2021-11    72933
2021-12    74922
2022-01    83107
2022-02    75693
2022-03    65961
have al        1
https:/        1
dtype: int64

In [8]:
# select tweets only from q1 2022
q1 = ['2022-03', '2022-02', '2022-01']
q1_2022 = df_tweets[df_tweets['tweet_yr_mth'].isin(q1)]

# q1_2022 = df_tweets.loc[df_tweets['tweet_yr_mth'] == ['2022-03', '2022-02', '2022-01']]

print(str(sum(q1_2022['Text'].map(len)))+ ' total characters across '+str(len(q1_2022))+' tweets in final set')

37100656 total characters across 224761 tweets in final set


In [9]:
q1_2022.groupby(['tweet_yr_mth']).size()

tweet_yr_mth
2022-01    83107
2022-02    75693
2022-03    65961
dtype: int64

# CATALOG

In [10]:
# load in catalog
# https://drive.google.com/file/d/1MRCN70GiMOah3XWyyz0s_vaDdB6DHaj6/view?usp=sharing
# df_cat = pd.read_csv('/Users/Mary/Desktop/NLP/Group Project/adbl_catalog_final.csv')

df_cat = pd.read_csv('/Users/creeg/Downloads/adbl_catalog_final.csv')
del df_cat["Unnamed: 0"] 


In [11]:
# df should have 203319 rows 
print(str(sum(df_cat['product_short_title'].map(len)))+ ' total book title characters and '+str(len(df_cat))+' books')
df_cat.drop(df_cat[df_cat['rating_count'] < 100].index, inplace=True)
print(str(sum(df_cat['product_short_title'].map(len)))+ ' total book title characters and '+str(len(df_cat))+' books after dropping < 100 review titles')


4945826 total book title characters and 203319 books
1441558 total book title characters and 67104 books after dropping < 100 review titles


In [12]:
df_cat.head()

,product_asin,product_short_title,product_author,release_date,rating,rating_count,legacy_genre,whitespace_ct,title_no_spaces,both_titles
0,B07FSNSLZ1,where the crawdads sing,Delia Owens,2018-08-14,4.760701,193799,Romance,3,wherethecrawdadssing,"('where the crawdads sing', 'wherethecrawdadss..."
1,B01I28NFEE,the subtle art of not giving a f*ck,Mark Manson,2016-09-13,4.516414,144510,"Relationships, Parenting & Personal Development",7,thesubtleartofnotgivingaf*ck,"('the subtle art of not giving a f*ck', 'thesu..."
2,B07KKMNZCH,cant hurt me,David Goggins,2018-11-28,4.874660,139548,"Relationships, Parenting & Personal Development",2,canthurtme,"('cant hurt me', 'canthurtme')"
3,B017V4IM1G,harry potter and the sorcerers stone,J.K. Rowling,2015-11-20,4.903521,139222,Science Fiction & Fantasy,5,harrypotterandthesorcerersstone,"('harry potter and the sorcerers stone', 'harr..."
4,B002UZZ93G,a game of thrones,George R.R. Martin,2003-12-09,4.757246,128418,Literature & Fiction,3,agameofthrones,"('a game of thrones', 'agameofthrones')"


In [13]:
# small bit of text cleaning that wasn't done when file was written
df_cat["product_short_title"] = df_cat["product_short_title"].str.replace("\'", "").str.strip() 

In [14]:
# delete and recreate title columns with newly cleaned text
del df_cat["title_no_spaces"] 
del df_cat["both_titles"] 
df_cat["title_no_spaces"] = df_cat["product_short_title"].str.replace(" ", "").str.strip() 
df_cat["both_titles"] = list(zip(df_cat['product_short_title'], df_cat['title_no_spaces']))

In [15]:
# create catalog dictionary
cat_dict = df_cat.set_index('product_asin').to_dict()['both_titles']

# ASIN to tweet assignment

In [16]:
tweets = q1_2022['Text'].to_list()

In [17]:
len(tweets)

224761

In [18]:
matched = {
    # For each key in the dictionary, create a new tuple
    key: tuple(
    # For the tweets, if there exists a string in the original tuple value that is a substring of that element, then retain it in the result.    
        filter(lambda x: any(substr in x for substr in value), tweets))
        for key, value in cat_dict.items()
}

# filter out all of the key-value pairs that have empty tuples for values using a dictionary comprehension. We could condense into one line, but I think it becomes pretty unreadable
result = {key: value for key, value in matched.items() if value != ()}


In [19]:
len(result)

13521

In [21]:
# write to json file
import json
with open('asin_to_tweet_assignment.json', 'w') as fp:
    json.dump(result, fp)